# Coherence Maximization Recipe Demo

The notebook describes how one can use the Coherence maximization recipe to build an experiment and train such a model which has high coherence value.

* Recipes in TopicNet: [recipes](https://github.com/machine-intelligence-laboratory/TopicNet/tree/master/topicnet/cooking_machine/recipes)
* Paper about coherence which is used in the recipe: [Intra-Text Coherence as a Measure of Topic Models' Interpretability](http://www.dialog-21.ru/media/4281/alekseevva.pdf)

# Contents<a id="contents"></a>

* [Loading Dataset](#data)
* [Formatting the Recipe](#formatting)
* [Experiment](#experiment)
* [Best Model](#best-model)
    * [Exploring Best Model](#investigating)

In [1]:
import os
import numpy as np
import shutil

import matplotlib.pyplot as plt
from matplotlib import cm

%matplotlib inline

In [2]:
from IPython.display import display_html

In [4]:
from topicnet.dataset_manager import load_dataset
from topicnet.cooking_machine import Dataset
from topicnet.cooking_machine.pretty_output import make_notebook_pretty
from topicnet.cooking_machine.recipes import IntratextCoherenceRecipe
from topicnet.viewers.top_tokens_viewer import TopTokensViewer
from topicnet.viewers.top_documents_viewer import TopDocumentsViewer

In [5]:
make_notebook_pretty()

## Loading Dataset<a id="data"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Let's pick up a dataset from the [list of available for download](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/dataset_manager/DemoDataset.ipynb)

In [6]:
DATASET_NAME = 'postnauka'

In [7]:
DATASET = load_dataset(DATASET_NAME)

In [8]:
DATASET._data.head()

,id,vw_text,raw_text
id,,,
1.txt,1.txt,1.txt |@author fuchs preobrazhensky tabachniko...,@title Автограф # «Математический дивертисмент...
2.txt,2.txt,2.txt |@word книга:2 лекция:3 рассматриваться:...,@title Главы: Маскулинности в российском конте...
3.txt,3.txt,3.txt |@word развитие появляться пиджина:4 бел...,@title Пиджины и креольские языки | @snippet Л...
4.txt,4.txt,4.txt |@word стандартный задача:3 состоять:4 р...,@title FAQ: Физиология микроводорослей | @snip...
5.txt,5.txt,5.txt |@2gramm повседневный_практика государст...,@title Русская государственная идеология | @sn...


In [9]:
DATASET_PATH = DATASET._data_path

In [10]:
DATASET_PATH

'/home/alekseev/topicnet/dataset_manager/postnauka.csv'

Let's see how big the dataset is

In [11]:
DATASET.get_dictionary()

artm.Dictionary(name=f0d91fae-b8ef-453f-a918-9a1a5b5a7bc4, num_entries=53095)

In [12]:
DATASET._data.shape

(3404, 3)

Nearly 3000 documents — seems not many.

## Formatting the Recipe<a id="formatting"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Let's use the flag below to reduce memory consumption in some places

In [13]:
LOW_MEMORY = True

Below is the main part: replacement of the placeholders in the recipe string with the real values.

In [14]:
training_pipeline = IntratextCoherenceRecipe()


num_documents_to_compute_coherence = 100  # not many — in order to speed up a bit 
total_num_documents = DATASET._data.shape[0]
documents_fraction = num_documents_to_compute_coherence / total_num_documents

recipe_config = training_pipeline.format_recipe(
    dataset_path=DATASET_PATH,
    main_modality='@word',
    modalities=['@word'],
    keep_dataset_in_memory=True,  # better try keep this True for quicker computation
    keep_dataset=not LOW_MEMORY,
    documents_fraction=documents_fraction, 
    num_specific_topics=20,
    num_background_topics=1,
    one_stage_num_iter=20,
    verbose=True,
)

In [15]:
print(recipe_config)

# The recipe mainly consists of basic cube stages,
# such as Decorrelation, Sparsing and Smoothing.
# In this way it is similar to ARTM baseline recipe.
# The core difference is that models selected based on their IntratextCoherenceScore
# (which is one of the scores included in TopicNet).
# PerplexityScore is also calculated to assure that models don't have high perplexity,
# but the main criteria is IntratextCoherenceScore.
#
# For more details about IntratextCoherence
# one may see the paper http://www.dialog-21.ru/media/4281/alekseevva.pdf
#
# Recipe usage sample:
#   file_contents_as_string = file_contents_as_string.format(
#     modality_names=modality_names,
#     main_modality=main_modality,
#     dataset_path=dataset_file_path,
#     keep_dataset_in_memory=True,
#     keep_dataset=False,
#     documents_fraction=documents_fraction,
#     specific_topics=specific_topic_names,
#     background_topics=background_topic_names,
#     one_stage_num_iter=20,
#     verbose=True,
#   )


## Experiment<a id="experiment"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Folder for future experiment

In [16]:
EXPERIMENTS_FOLDER_PATH = os.path.join('..', 'experiments')

In [17]:
os.makedirs(EXPERIMENTS_FOLDER_PATH, exist_ok=True)

In [18]:
os.listdir(EXPERIMENTS_FOLDER_PATH)

['Maximize_Coherence2', 'Maximize_Coherence']

In [19]:
EXPERIMENT_ID = 'Maximize_Coherence'

Removing the folder with the chosen experiment ID if it exists

In [20]:
COHERENCE_EXPERIMENT_FOLDER_PATH = os.path.join(EXPERIMENTS_FOLDER_PATH, EXPERIMENT_ID)

if os.path.isdir(COHERENCE_EXPERIMENT_FOLDER_PATH):
    shutil.rmtree(COHERENCE_EXPERIMENT_FOLDER_PATH)

In [21]:
%%time

experiment, dataset = training_pipeline.build_experiment_environment(
    experiment_id=EXPERIMENT_ID,
    save_path=EXPERIMENTS_FOLDER_PATH,
)

CPU times: user 2.85 s, sys: 294 ms, total: 3.14 s
Wall time: 3.01 s


In [22]:
dataset.get_dictionary()

artm.Dictionary(name=db8c7870-be41-411f-9dc1-c28d6d789fd6, num_entries=53095)

There are too many words in the dictionary.
Let's filter it (otherwise Phi matrix will be very huge).

In [23]:
dictionary = dataset.get_dictionary()
dictionary.filter(min_df_rate=0.01, max_df_rate=0.9)
dataset._cached_dict = dictionary

In [24]:
dataset.get_dictionary()

artm.Dictionary(name=db8c7870-be41-411f-9dc1-c28d6d789fd6, num_entries=5441)

Experiment's `low_memory` mode:

In [25]:
experiment._low_memory = LOW_MEMORY

In [26]:
experiment._low_memory

True

Go!

In [27]:
%%time

experiment.run(dataset)

100%|██████████| 100/100 [00:26<00:00,  3.82it/s]


../../topicnet/cooking_machine/cubes/base_cube.py:219: UserWarning:

Perplexity is too high for threshold 1.05



100%|██████████| 100/100 [00:27<00:00,  3.64it/s]


../../topicnet/cooking_machine/cubes/base_cube.py:219: UserWarning:

Max progression length exceeded



100%|██████████| 100/100 [00:24<00:00,  4.00it/s]


../../topicnet/cooking_machine/models/dummy_topic_model.py:160: UserWarning:

Already dummy



100%|██████████| 100/100 [00:25<00:00,  3.99it/s]
CPU times: user 2min 45s, sys: 2min 52s, total: 5min 37s
Wall time: 29min 3s


{Model(id=--10h05m21s_23d04m2020y---, parent_id=--10h02m48s_23d04m2020y---, experiment_id=Maximize_Coherence)}

## Best Model<a id="best-model"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Let's find the best model of all (not only from the last stage of the experiment).

In [28]:
SCORE_NAME = 'IntratextCoherenceScore'

BEST_MODEL = None
levels = range(1, len(experiment.cubes) + 1)

for level in levels:
    best_model_candidates = experiment.select(
        f'{SCORE_NAME} -> max',
        level=level
    )

    if len(best_model_candidates) == 0:
        continue

    best_model_candidate = best_model_candidates[0]

    if (BEST_MODEL is None or
            BEST_MODEL.scores[SCORE_NAME][-1] <
            best_model_candidate.scores[SCORE_NAME][-1]):

        BEST_MODEL = best_model_candidate

../../topicnet/cooking_machine/routine.py:455: UserWarning:

Model "Model(id=-----------root-----------, parent_id=None, experiment_id=Maximize_Coherence)" has empty value list for score "IntratextCoherenceScore"

../../topicnet/cooking_machine/routine.py:484: UserWarning:

Can't return the requested number of models:



In [29]:
BEST_MODEL

Model(id=--10h05m21s_23d04m2020y---, parent_id=--10h02m48s_23d04m2020y---, experiment_id=Maximize_Coherence)

All other models (excluding the Root one):

In [30]:
models_to_compare_with = [
    m for m in experiment.models.values()
    if len(m.scores[SCORE_NAME]) > 0
]

In [31]:
len(models_to_compare_with)

23

In [32]:
len(experiment.models)  # plus Root

24

Best model's ID and its score value

In [33]:
print(f'         ID: {BEST_MODEL.model_id}')
print(f'Score value: {BEST_MODEL.scores[SCORE_NAME][-1]:.5f}')

         ID: --10h05m21s_23d04m2020y---
Score value: 0.01554


Other models' score value:

In [34]:
for level in levels:
    print('Level:', level)
    
    current_models = experiment.select(
        '',
        level=level
    )
    
    for m in current_models:
        if len(m.scores[SCORE_NAME]) == 0:
            score_value = float('nan')
        else:
            score_value = m.scores[SCORE_NAME][-1]

        print(f'{m.model_id}: {score_value:.5f}')
    
    print()

Level: 1
-----------root-----------: nan

Level: 2
--09h40m13s_23d04m2020y---: 0.00135
--09h41m07s_23d04m2020y---: 0.00068
--09h42m03s_23d04m2020y---: 0.00068
--09h43m01s_23d04m2020y---: 0.00069
--09h44m02s_23d04m2020y---: 0.00081
--09h45m06s_23d04m2020y---: 0.00054
--09h48m50s_23d04m2020y---: 0.00013

Level: 3
--09h50m34s_23d04m2020y---: 0.00180
--09h51m33s_23d04m2020y---: 0.00180
--09h52m34s_23d04m2020y---: 0.00450
--09h53m37s_23d04m2020y---: 0.00851
--09h54m41s_23d04m2020y---: 0.00988
--09h55m49s_23d04m2020y---: 0.01175

Level: 4
--09h57m16s_23d04m2020y---: 0.00991
--09h58m18s_23d04m2020y---: 0.00991
--09h59m22s_23d04m2020y---: 0.01048

Level: 5
--10h00m36s_23d04m2020y---: 0.01036
--10h01m41s_23d04m2020y---: 0.01091
--10h02m48s_23d04m2020y---: 0.01231
--10h03m57s_23d04m2020y---: 0.00000

Level: 6
--10h05m21s_23d04m2020y---: 0.01554
--10h06m33s_23d04m2020y---: 0.01554
--10h07m47s_23d04m2020y---: 0.01554



## Exploring Best Model<a id="investigating"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

More detailed description of the best model goes here.

In [35]:
BEST_MODEL.describe_regularizers()

tau  gamma class_ids
model_id                   regularizer_name                                 
--10h05m21s_23d04m2020y--- decorrelate_phi         0.050000    0.0        []
                           smooth_phi_background  27.321693    NaN        []
                           smooth_phi_bcg          0.000000    NaN   [@word]
                           smooth_phi_specific     0.000000    NaN        []
                           smooth_theta_bcg        0.000000    NaN      None
                           sparse_phi_specific    -2.837010    NaN        []

In [36]:
BEST_MODEL.describe_scores()

last_value
model_id                   score_name                                   
--10h05m21s_23d04m2020y--- PerplexityScore@all                   3795.78
                           SparsityThetaScore                    0.32957
                           SparsityPhiScore@word                0.912853
                           PerplexityScore@word                  3795.78
                           TopicKernel@word.average_coherence          0
                           TopicKernel@word.average_contrast     0.64136
                           TopicKernel@word.average_purity      0.856669
                           TopicKernel@word.average_size         1151.43
                           IntratextCoherenceScore             0.0155387

Let's use some viewers:

In [37]:
toptok_viewer = TopTokensViewer(BEST_MODEL, num_top_tokens=10, method='phi')
topdoc_viewer = TopDocumentsViewer(BEST_MODEL, dataset=dataset)

In [41]:
top_tokens_view = toptok_viewer.view_from_jupyter(
    display_output=False,
    give_html=True,
)

In [42]:
display_html(' '.join(top_tokens_view), raw=True)

topic_0 modality token 
 
 @word 
 работать 
 0.042580 
 
 
 наука 
 0.042490 
 
 
 хороший 
 0.036730 
 
 
 учёный 
 0.030780 
 
 
 заниматься 
 0.024270 
 
 
 хотеть 
 0.018140 
 
 
 думать 
 0.018000 
 
 
 университет 
 0.017760 
 
 
 образование 
 0.017110 
 
 
 проект 
 0.016900 
 
 topic_1 modality token 
 
 @word 
 говорить 
 0.010790 
 
 
 работа 
 0.008430 
 
 
 являться 
 0.008020 
 
 
 образ 
 0.007370 
 
 
 вопрос 
 0.007150 
 
 
 сторона 
 0.006980 
 
 
 должный 
 0.006950 
 
 
 жизнь 
 0.006710 
 
 
 важный 
 0.005940 
 
 
 существовать 
 0.005900 
 
 topic_2 modality token 
 
 @word 
 сантиметр 
 0.240490 
 
 
 святилище 
 0.097950 
 
 
 сосуд 
 0.063960 
 
 
 стоящий 
 0.063510 
 
 
 заполнить 
 0.050610 
 
 
 культовый 
 0.044140 
 
 
 фигурка 
 0.044060 
 
 
 квадрат 
 0.040780 
 
 
 статуэтка 
 0.030470 
 
 
 ниша 
 0.025790 
 
 topic_3 modality token 
 
 @word 
 книга 
 0.058980 
 
 
 история 
 0.048080 
 
 
 текст 
 0.029040 
 
 
 писать 
 0.027460 
 
 
 автор 
 0.026690 
 
 
 метр 
 0.025300 
 
 
 написать 
 0.019160 
 
 
 литература 
 0.014750 
 
 
 читать 
 0.012740 
 
 
 искусство 
 0.011840 
 
 topic_4 modality token 
 
 @word 
 россия 
 0.019280 
 
 
 государство 
 0.014590 
 
 
 общество 
 0.013680 
 
 
 право 
 0.013230 
 
 
 власть 
 0.012100 
 
 
 политический 
 0.010340 
 
 
 война 
 0.009390 
 
 
 политика 
 0.007500 
 
 
 закон 
 0.006790 
 
 
 советский 
 0.005780 
 
 topic_5 modality token 
 
 @word 
 caption 
 0.103410 
 
 
 остров 
 0.069080 
 
 
 align 
 0.051350 
 
 
 width 
 0.051350 
 
 
 attachment 
 0.050610 
 
 
 африка 
 0.038090 
 
 
 глубина 
 0.035500 
 
 
 раса 
 0.033250 
 
 
 гора 
 0.032760 
 
 
 aligncenter 
 0.028910 
 
 topic_6 modality token 
 
 @word 
 язык 
 0.081550 
 
 
 слово 
 0.066310 
 
 
 память 
 0.018430 
 
 
 речь 
 0.014260 
 
 
 вариант 
 0.013940 
 
 
 информация 
 0.013720 
 
 
 ошибка 
 0.012100 
 
 
 понятно 
 0.011540 
 
 
 рука 
 0.010060 
 
 
 предложение 
 0.009770 
 
 topic_7 modality token 
 
 @word 
 лёд 
 0.143540 
 
 
 климат 
 0.125500 
 
 
 морской 
 0.114530 
 
 
 ледник 
 0.088150 
 
 
 накапливаться 
 0.082020 
 
 
 антарктида 
 0.043160 
 
 
 дно 
 0.040830 
 
 
 канадский 
 0.037540 
 
 
 ледяной 
 0.035190 
 
 
 арктика 
 0.030680 
 
 topic_8 modality token 
 
 @word 
 женщина 
 0.041210 
 
 
 животное 
 0.032460 
 
 
 эволюция 
 0.023740 
 
 
 мужчина 
 0.021630 
 
 
 растение 
 0.019880 
 
 
 самец 
 0.012820 
 
 
 птица 
 0.012430 
 
 
 самка 
 0.011450 
 
 
 животный 
 0.010780 
 
 
 признак 
 0.010620 
 
 topic_9 modality token 
 
 @word 
 выбрать 
 0.000000 
 
 
 датировка 
 0.000000 
 
 
 колоть 
 0.000000 
 
 
 марков 
 0.000000 
 
 
 необходимый 
 0.000000 
 
 
 обезболивать 
 0.000000 
 
 
 перестроиться 
 0.000000 
 
 
 подруга 
 0.000000 
 
 
 пьяный 
 0.000000 
 
 
 сартр 
 0.000000 
 
 topic_10 modality token 
 
 @word 
 система 
 0.010830 
 
 
 получить 
 0.007110 
 
 
 большой 
 0.006900 
 
 
 частица 
 0.006240 
 
 
 звезда 
 0.005440 
 
 
 метод 
 0.005390 
 
 
 происходить 
 0.005240 
 
 
 земля 
 0.005190 
 
 
 эксперимент 
 0.005160 
 
 
 вселенная 
 0.004860 
 
 topic_11 modality token 
 
 @word 
 лекция 
 0.101740 
 
 
 сеть 
 0.089110 
 
 
 интернет 
 0.068310 
 
 
 num 
 0.060790 
 
 
 pcourse 
 0.060790 
 
 
 процедура 
 0.043950 
 
 
 пользователь 
 0.033370 
 
 
 страница 
 0.030230 
 
 
 узел 
 0.022500 
 
 
 al 
 0.022020 
 
 topic_12 modality token 
 
 @word 
 территория 
 0.010040 
 
 
 бог 
 0.009990 
 
 
 традиция 
 0.009120 
 
 
 народ 
 0.008750 
 
 
 имя 
 0.008050 
 
 
 культура 
 0.007090 
 
 
 миф 
 0.006980 
 
 
 век 
 0.006250 
 
 
 эпоха 
 0.006090 
 
 
 церковь 
 0.006010 
 
 topic_13 modality token 
 
 @word 
 стандарт 
 0.112860 
 
 
 прогноз 
 0.098000 
 
 
 станция 
 0.068200 
 
 
 сдвиг 
 0.064730 
 
 
 геологический 
 0.059970 
 
 
 землетрясение 
 0.052460 
 
 
 протокол 
 0.050560 
 
 
 слоить 
 0.047970 
 
 
 обстановка 
 0.042370 
 
 
 дополнение 
 0.041750 
 
 topic_14 modality toke

Parts of topics' top documents:

In [45]:
topdoc_viewer.view_from_jupyter(
    current_num_top_doc=5,
    num_view_topics=5,
)

Topic name: topic_0 8.txt @title Перспективы: Квантовые технологии | @snippet Физик Александр Львовский в новой программе «Перспективы» рассказывает о Российском Квантовом Центре, «Star Trek» и квантовой криптографии
. 2170.txt @title Перспективы: Философия массовой культуры | @snippet Философ Александр Павлов в новой программе «Перспективы» рассказывает об эпохе постмодерна, философии «Симпсонов», либертарианстве и Славое Жижеке
. 1552.txt @title Перспективы: Социология академической жизни | @snippet Социолог Михаил Соколов в новой программе «Перспективы» рассказывает о социологии науки, деревенских дисциплинах и академической репутации
. 405.txt @title Перспективы: Доказательная медицина | @snippet Кардиолог Ярослав Ашихмин в новой программе «Перспективы» рассказывает о защите диссертаций, русской медицинской школе и новом поколении врачей
. 3194.txt @title Перспективы: Квантовая оптика и квантовая телепортация | @snippet Физик Юджин Ползик в новой программе «Перспективы» рассказывает о механизме телепортации, информационной безопасности и квантовом компьютере
. Topic name: topic_1 19.txt @title Социология моды | @snippet Социолог Александр Гофман о стандартах поведения, атрибутивных ценностях и ситуации двойного обозначения
. 3209.txt @title Социология маргинальности | @snippet Социолог Светлана Баньковская об отклонении от нормы участия, положении чужака в группе и практикуемой маргинальности
. 922.txt @title Мультикультурализм | @snippet Политолог Владимир Малахов об отказе от национального государства, статусе русского языка и двух уровнях мультикультурализма
. 1205.txt @title Восприятие реальности | @snippet Социолог Виктор Вахштайн о джеймсовской проблеме, правилах Макнотена и мире верховной реальности
. 2037.txt @title Повседневное событие | @snippet Социолог Виктор Вахштайн о транспонированной форме взаимодействия, феномене нетранзитивности и значении физического контекста
. Topic name: topic_2 Topic name: topic_3 336.txt @title Литературный мир «Хоббита» | @snippet Филолог Мария Штейнман о литературной игре, особенностях стиля Толкина и трех уровнях «Хоббита»
. 2933.txt @title Детский фольклор | @snippet Фольклорист Михаил Алексеевский о всплеске интереса к детскому фольклору, о его жанрах и об отношении к нему в советское время.
. 1895.txt @title Образ Жанны д'Арк | @snippet Историк Ольга Тогоева о святой и ведьме, исторической интерпретации образа и одержимости дьяволом
. 1607.txt @title Документальное театроведение | @snippet Искусствовед Алексей Бартошевич о закрытых архивах, двухтомнике Вахтангова и новой театральной концепции
. 2267.txt @title Осип Мандельштам | @snippet Литературовед Олег Лекманов о воспоминаниях Надежды Мандельштам, понятии «подтекст» и подмене статьи Лидии Гинзбург
. Topic name: topic_4 1853.txt @title Завершение Французской революции XVIII века | @snippet Историк Дмитрий Бовыкин о термидорианском перевороте, принятии Конституции III года и «политике качелей»
. 2866.txt @title Суд и следствие над декабристами | @snippet Историк Ольга Эдельман о создании III отделения, кодификации законов и приговорах участникам восстания
. 836.txt @title Последствия Первой мировой войны | @snippet Историк Илья Женин о последних годах войны, Версальском мирном договоре и судьбе союзников Германии
. 873.txt @title Веймарская республика | @snippet Историк Илья Женин о республике без республиканцев, Октябрьской революции и Социал-демократической партии
. 3384.txt @title Феномен внезапности во Второй мировой войне | @snippet Историк Алексей Киличенков о геополитической ситуации накануне Второй мировой, польской разведке и авантюризме Гитлера
.

Some topics (eg. `topic_2`) has no documents: these topics have empty rows in $\Theta$ matrix.